In [1]:
from fitting import FitModel
from databases import initiate_db, print_summary
from plotters import forces_plots
from scipy import optimize
import numpy as np
import matplotlib.pyplot as plt

In [2]:
params = {}
params['core_shell'] = { 'Li': False, 'Ni': False, 'O': False }
params['charges'] = {'Li': +1.0,
                     'Ni': +3.0,
                     'O': -2.0}
params['masses'] = {'Li': 6.941,
                    'Ni': 58.6934,
                    'O': 15.999}
params['cs_springs'] = {}


# params['core_shell'] = { 'Li': False, 'Ni': False, 'O': True }
# params['charges'] = {'Li': +1.0,
#                      'Ni': +3.0,
#                      'O': {'core':  +0.960,
#                            'shell': -2.960}}
# params['masses'] = {'Li': 6.941,
#                     'Ni': 58.6934,
#                     'O': {'core': 14.3991,
#                           'shell': 1.5999} }
# params['cs_springs'] = {'O' : [100.0, 0.0]}


distribution = {}
distribution['Li-O'] = {'bpp' : [663.111, 0.119, 0.0],
                        'sd' : [80, 0.01, 0.01]}

distribution['Ni-O'] = {'bpp' : [1393.540, 0.218, 0.000],
                        'sd'  : [80, 0.01, 0.01]}

distribution['O-O'] = {'bpp' : [25804.807, 0.284, 0.0],
                       'sd'  : [200, 0.01, 5]}
       
excude_from_fit = [] # string of atom1_atom2_param. Example of format = 'O_O_rho'

In [3]:
fit_data = FitModel.collect_info(params, distribution, supercell=[1,1,1])

Found elements: ['Li', 'Ni', 'O']


In [4]:
include_labels = ['Li_O_a', 'Li_O_rho', 'Ni_O_a', 'Ni_O_rho', 'O_O_a', 'O_O_rho']
# include_values = [1.06819131e+03, 2.11191786e-01, 3.10582127e+02, 2.60191312e-01, 2.89687978e+04, 2.81854872e-01]
bounds_list = [(100.0,2000.0),(0.01,1.0),(100.0,3000.0),(0.01,1.0),(100.0,40000.0),(0.01,1.0)]

In [5]:
# optimize.minimize(fit_data.fit_error, include_values, args=(include_labels), method='TNC', bounds=bounds_list, options={'maxiter':3000, 'maxfev':3000, })

In [6]:
# optimize.minimize(fit_data.fit_error, include_values, args=(include_labels), method='trust-constr', bounds=bounds_list, options={'xtol':0.001})

In [7]:
s = optimize.differential_evolution(fit_data.fit_error,
                                    bounds=bounds_list,
                                    popsize=15,
#                                     tol=0.0001,
                                    args=([include_labels]),
                                    maxiter=1000,
                                    disp=True)

differential_evolution step 1: f(x)= 6.22717
differential_evolution step 2: f(x)= 3.80092
differential_evolution step 3: f(x)= 3.80092
differential_evolution step 4: f(x)= 3.80092
differential_evolution step 5: f(x)= 3.80092
differential_evolution step 6: f(x)= 3.80092
differential_evolution step 7: f(x)= 3.80092
differential_evolution step 8: f(x)= 3.80092
differential_evolution step 9: f(x)= 3.80092
differential_evolution step 10: f(x)= 3.80092
differential_evolution step 11: f(x)= 3.80092
differential_evolution step 12: f(x)= 3.78254
differential_evolution step 13: f(x)= 3.78254
differential_evolution step 14: f(x)= 3.78254
differential_evolution step 15: f(x)= 3.78254
differential_evolution step 16: f(x)= 3.78254
differential_evolution step 17: f(x)= 3.78254
differential_evolution step 18: f(x)= 3.78016
differential_evolution step 19: f(x)= 3.78016
differential_evolution step 20: f(x)= 3.75214
differential_evolution step 21: f(x)= 3.75214
differential_evolution step 22: f(x)= 3.730

In [8]:
s

     fun: 3.730229774096095
 message: 'Optimization terminated successfully.'
    nfev: 3517
     nit: 38
 success: True
       x: array([9.69232104e+02, 1.18155369e-01, 1.67231182e+02, 5.61912419e-01,
       2.77242783e+04, 4.45542929e-02])

In [ ]:
def fit_forces(fit_data, values, args):
    potential_params = dict(zip( args, values ))
    for pot in fit_data.potentials:
        for param in [ pot.a, pot.rho, pot.c ]:
            key = param.label_string
            if key not in potential_params:
                potential_params[key] = param.value
    ip_forces = fit_data.get_forces(**potential_params)
    return ip_forces

In [ ]:
include_values = s.x
ip_list = np.concatenate(fit_forces(fit_data, include_values, include_labels), axis=0)
dft_list = np.concatenate(fit_data.expected_forces(), axis=0)

In [ ]:
plt.plot(ip_list, label='ip')
plt.plot(dft_list, label='dft')
plt.ylabel('force')
plt.xlabel('index')
plt.legend()
# plt.savefig('global_minimiser_example.png',dpi=500, bbox_inches = "tight")
plt.show()
np.sum(((dft_list-ip_list)**2)/ip_list.size)

Get Lattice Parameters

In [ ]:
import lammps
def get_lattice(fit_data):
    potential_params = {}
    for pot in fit_data.potentials:
        for param in [pot.a, pot.rho, pot.c]:
            key = param.label_string
            potential_params[key] = param.value
    lmp = fit_data.get_lattice_params(**potential_params)
    return lmp

In [ ]:
lmp = get_lattice(fit_data)
lammps = lmp[0]

In [ ]:
a_c_vol = [(lammps.box.lengths[0]/8),(lammps.box.lengths[2]/2),lammps.box.volume/8/4/2/2]
ref_delmas = [2.879, 14.201, 101.951]

In [ ]:
a_dif = (a_c_vol[0]-ref_delmas[0])/((a_c_vol[0]+ref_delmas[0])/2) * 100
c_dif = (a_c_vol[1]-ref_delmas[1])/((a_c_vol[1]+ref_delmas[1])/2) * 100
vol_dif = (a_c_vol[2]-ref_delmas[2])/((a_c_vol[2]+ref_delmas[2])/2) * 100

In [ ]:
print('My calculated values are (a,c,vol): {}'.format(a_c_vol))
print('Experimental values are (a,c,vol): {}'.format(ref_delmas))
print('Percentage differences are (a,c,vol): {}, {}, {}'.format(a_dif, c_dif, vol_dif))

In [ ]:
ref_DFT = [2.729, 14.053, 96.456]

In [ ]:
a_dif = (a_c_vol[0]-ref_DFT[0])/((a_c_vol[0]+ref_DFT[0])/2) * 100
c_dif = (a_c_vol[1]-ref_DFT[1])/((a_c_vol[1]+ref_DFT[1])/2) * 100
vol_dif = (a_c_vol[2]-ref_DFT[2])/((a_c_vol[2]+ref_DFT[2])/2) * 100

In [ ]:
print('My calculated values are (a,c,vol): {:.3f} {:.3f} {:.3f}'.format(a_c_vol[0], a_c_vol[1], a_c_vol[2]))
print('DFT values are (a,c,vol): {:.3f} {:.3f} {:.3f}'.format(ref_DFT[0], ref_DFT[1], ref_DFT[2]))
print('Percentage differences are (a,c,vol): {:.3f} {:.3f} {:.3f}'.format(a_dif, c_dif, vol_dif))

In [ ]:
lammps.box.lengths

In [ ]:
# Geometry optimised structure:
#      fun: 0.00024535877391032265
#  message: 'Optimization terminated successfully.'
#     nfev: 14587
#      nit: 161
#  success: True
#        x: array([6.63110880e+02, 1.19385635e-01, 1.39354000e+03, 2.17550689e-01,
#        2.58048072e+04, 2.84121273e-01])

In [ ]:
# rigid ion with C=0:
#     My calculated values are (a,c,vol): 2.853 15.056 117.474
#     DFT values are (a,c,vol): 2.729 14.053 96.456
#     Percentage differences are (a,c,vol): 4.430 6.893 19.649

In [ ]:
# rigid ion with O-O C=27.1:
#     My calculated values are (a,c,vol): 2.834 15.203 117.767
#     DFT values are (a,c,vol): 2.729 14.053 96.456
#     Percentage differences are (a,c,vol): 3.763 7.864 19.896

In [ ]:
# rigid ion with O-O C=45.0:
#     My calculated values are (a,c,vol): 2.771 15.306 120.610
#     DFT values are (a,c,vol): 2.729 14.053 96.456
#     Percentage differences are (a,c,vol): 1.531 8.539 22.255

In [ ]:
# core-shell with C=0:
#     My calculated values are (a,c,vol): 2.857 14.945 115.741
#     DFT values are (a,c,vol): 2.729 14.053 96.456
#     Percentage differences are (a,c,vol): 4.572 6.151 18.176

In [ ]:
# core-shell with C=27.1:
#     My calculated values are (a,c,vol): 2.819 15.140 117.582
#     DFT values are (a,c,vol): 2.729 14.053 96.456
#     Percentage differences are (a,c,vol): 3.234 7.445 19.740

In [ ]:
# core-shell with C=45.0:
#     My calculated values are (a,c,vol): 2.818 15.119 116.555
#     DFT values are (a,c,vol): 2.729 14.053 96.456
#     Percentage differences are (a,c,vol): 3.225 7.310 18.872